In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from time import sleep
import os
from dotenv import load_dotenv
from datetime import datetime

# Cargamos variables de entorno
load_dotenv()

# Inicializamos el WebDriver
driver = webdriver.Chrome()

# Configuramos el tiempo de espera implícito a 10 segundos
driver.implicitly_wait(10)

# URL de la web de hoteles
# En este caso la url ya lleva las selecciones de fecha y disponibilidad
url_hoteles = os.getenv("url_hoteles")

# Abrimos la url
driver.get(url_hoteles)
driver.maximize_window()
sleep(10)  # Pausa para permitir la carga completa de la página


# Encuentra todos los bloques de información de los hoteles en la página
hotel_blocks = driver.find_elements(By.CSS_SELECTOR, ".hotelblock__content")

# Lista vacía para almacenar la información de los hoteles
hoteles_info = []

# Itera sobre cada bloque de hotel y extrae la información
for hotel in hotel_blocks:
    nombre_hotel = (hotel.find_element(By.CSS_SELECTOR, ".title__link").text).split('\n')[0] # split('\n')[0] para obtener solo el nombre del hotel
    estrellas = (hotel.find_element(By.CSS_SELECTOR, ".ratings__score").text).split('/')[0] # obtener solo el número de estrellas
    # Como hay hoteles con precios para socios, queremos obtener el precio por defecto
    try: # intentamos obtener el precio por defecto
        precio = hotel.find_element(By.CSS_SELECTOR, ".rate-details__price2__first-price.rate-details__price--default .booking-price__number.mcp-price-number").text
        
    except: # si solo hay un precio nos devuelve el único disponible
        precio = hotel.find_element(By.CSS_SELECTOR, ".rate-details__price1__first-price.rate-details__price--default .booking-price__number.mcp-price-number").text
    #Sacamos la fecha del scrapeo que será la fecha de reserva
    fecha_scrapeo = datetime.now().date()       
# añadimos la información a la lista
    hoteles_info.append({
        "nombre": nombre_hotel,
        "estrellas": estrellas,
        "precio": precio,
        "fecha_reserva" : fecha_scrapeo
    })
# Cerrar el navegador
driver.quit()
# Crear un dataframe con la información obtenida
df_hoteles_competencia = pd.DataFrame(hoteles_info)
df_hoteles_competencia.to_pickle('hoteles_competencia.pkl')
print("df guardado")


df guardado
